In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/28 19:59:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/28 19:59:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/02/28 19:59:26 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [22]:
spark.version

'3.5.0'

In [3]:
df = spark.read \
.option("header", "true") \
.csv('fhv_tripdata_2019-10.csv')

In [7]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [4]:
!head -n 1001 fhv_tripdata_2019-10.csv > head_hw.csv

In [5]:
import pandas as pd
df_pandas = pd.read_csv('head_hw.csv')

In [6]:
df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [8]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [10]:
from pyspark.sql import types

In [11]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),
    types.StructField('PUlocationID', types.IntegerType(), True),
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
    ])

In [12]:
df = spark.read \
	.option("header", "true") \
	.schema(schema) \
	.csv('fhv_tripdata_2019-10.csv')

In [13]:
df.repartition(6).write.parquet('data/pq/fhv')

In [17]:
df = spark.read.parquet('data/pq/fhv/*')

In [18]:
df.createOrReplaceTempView('fhv_trip_data')

In [20]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01537|2019-10-02 07:09:00|2019-10-02 07:23:00|         264|         264|   NULL|                B01537|
|              B00860|2019-10-02 01:58:19|2019-10-02 02:16:59|         264|         127|   NULL|                B00860|
|              B01016|2019-10-02 05:47:05|2019-10-02 05:54:18|         264|          77|   NULL|                B01016|
|              B03164|2019-10-03 07:26:48|2019-10-03 07:39:07|         264|          20|   NULL|                B03164|
|              B01984|2019-10-02 10:33:00|2019-10-02 11:05:00|         264|          36|   NULL|                B01984|
|              B00906|2019-10-01 05:08:5

In [21]:
spark.sql("""
    SELECT COUNT(1) as number_trips
    FROM fhv_trip_data
    WHERE date_trunc('day', pickup_datetime) = '2019-10-15'
          """).show()

+------------+
|number_trips|
+------------+
|       62610|
+------------+



In [28]:
spark.sql("""
    SELECT MAX(
                TIMESTAMPDIFF(
                    HOUR,
                    pickup_datetime,
                    dropOff_datetime
                    )
                )
    FROM fhv_trip_data
          """).show()

+-----------------------------------------------------------+
|max(timestampdiff(HOUR, pickup_datetime, dropOff_datetime))|
+-----------------------------------------------------------+
|                                                     631152|
+-----------------------------------------------------------+



In [29]:
df_zones = spark.read.parquet('zones/*')

In [30]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [34]:
df_join = df.join(df_zones, df.PUlocationID == df_zones.LocationID)

In [35]:
df_join.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+--------+--------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID| Borough|          Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+--------+--------------+------------+
|              B01537|2019-10-02 07:09:00|2019-10-02 07:23:00|         264|         264|   NULL|                B01537|       264| Unknown|            NV|         N/A|
|              B00860|2019-10-02 01:58:19|2019-10-02 02:16:59|         264|         127|   NULL|                B00860|       264| Unknown|            NV|         N/A|
|              B01016|2019-10-02 05:47:05|2019-10-02 05:54:18|         264|          77|   NULL|                B01016|       264| Unknown|            NV|      

In [36]:
df_join.drop('LocationID').createOrReplaceTempView('fhv_zone_data')

In [49]:
spark.sql("""
          WITH trips_by_zone AS
          (SELECT Zone, COUNT(*) AS num_trips
          FROM fhv_zone_data
          GROUP BY Zone)
          SELECT Zone, num_trips
          FROM trips_by_zone
          ORDER BY num_trips ASC
          LIMIT 1
          """).show()

+-----------+---------+
|       Zone|num_trips|
+-----------+---------+
|Jamaica Bay|        1|
+-----------+---------+



24/02/29 04:19:15 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1065226 ms exceeds timeout 120000 ms
24/02/29 04:19:15 WARN SparkContext: Killing executors is not supported by current scheduler.
24/02/29 04:35:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$